## GBDT & XGBoost

$$ 
CART
\xrightarrow[]{\text{fit residual term, SSE loss}}
BoostingTree 
\xrightarrow[]{\text{generalized loss-function} \quad  L(\cdot)}
GBDT  
\xrightarrow[\text{pruning, NA, parallel, etc.}]{ \text{add regulation term} \quad \gamma T + \frac{1}{2}\|w\|^2}
XGBoost
$$

### GBDT

$L(\cdot)$不再具有平方误差的具体形式，而是一般化的损失函数。

使用 <u>**损失函数的负梯度 在当前模型的值**</u> 作为残差的替代

$$ r_{mi} = - \frac{\partial L(y_i, \hat{y}_i)}{\partial \hat{y}_i} \Big|_{ \hat{y}_i = f_{m-1}(x_i) }   $$

下一个基回归树对 $r_{mi}$ 进行拟合

### XGBoost

加入正则项

$$
\mathcal{L} = \sum_i L(y_i, \hat{y}_i) + \sum_k \Omega(f_k)
$$

其中  $$ \Omega(\cdot) = \gamma T + \frac{\lambda}{2} \|w\|^2 $$ 是单棵树复杂度的度量

加法模型 && 前向分步算法，第 $t$ 步的目标函数为

$$ \mathcal{L}^{(t)} = \sum_i L\big(y_i, \quad \hat{y}_i^{(t-1)} + f_t(x_i) \big) + \Omega(f_t) + Constant$$

（注意优化的自变量为 $f_t$）

将 $\Omega(f_t)$ 展开，得到

$$ 
\mathcal{L}^{(t)} = \sum_i L(y_i, \hat{y}_i^{(t-1)} + f_t(x_i) ) + \gamma T + \frac{\lambda}{2} \sum_{j=1}^T w_j^2
$$

在 $\hat{y}_i^{(t-1)}$ 处展开 （同GBDT，在 **当前模型的值** 处展开），展开步长为 $f_t(x_i)$ 

> [这个回答](https://www.zhihu.com/question/63560633/answer/375811211)
> 
> 二阶泰勒展开
$$ f(x + \Delta x) = f(x) + f^\prime(x) \Delta x + \frac{1}{2} f^{\prime\prime} (x) \Delta x^2$$
这里 x 为当前模型输出值 $\hat{y}_i^{(t-1)}$， 而 $\Delta x$ 为本次新加入的树 $f_t(x_i)$

展开后的目标函数

$$ 
\mathcal{L}^{(t)} = \sum_i \Big[ L(y_i, \hat{y}_i^{(t-1)}) + g_i f_t(x_i) + \frac{1}{2} h_i f_t(x_i)^2 \Big]  
+ \gamma T + \frac{\lambda}{2} \sum_{j=1}^T w_j^2
$$

其中 $g_i, h_i$ 为 $L$ 的一阶导数、二阶导数 在 $(y_i, \hat{y}_i^{(t-1)})$  处的值

中括号中的第一项也是Constant，可以忽略；

重新排列组合，按照叶子节点重新组合，注意 $f_t(x_i)$ 一定属于某个 $R_j$ 从而等于某个 $wj$

$$ \mathcal{L}^{(t)} = \sum_{j=1}^T \Big[ g_j w_j + \frac{h_j + \lambda}{2} w_j^2 \Big] + \gamma T$$

$$\text{where} \quad g_j = \sum_{x_i \in R_j} g_i , \quad h_j = \sum_{x_i \in R_j} h_i $$

这是一个关于 $w_j$ 的一元二次函数，根据顶点公式

$$ w_j^* =  - \frac{b}{2a} = - \frac{g_j}{h_j + \lambda} $$

$$ Obj^* =  \frac{4ac - b^2}{4a} = - \frac{1}{2} \sum_{j=1}^T \frac{g_j^2}{(h_j + \lambda)} + \gamma T$$

<u> 上述逻辑解决的问题是“已知树的结构，如何确定w”，现在的问题是如何确定树结构 ？</u>